In [ ]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
from datasets import load_dataset, Dataset

# insert your code here
dataset = load_dataset("yelp_polarity")


Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
# insert your code here
import pandas as pd

train_labels = set(dataset['train']['label'])
test_labels = set(dataset['test']['label'])

# Display the unique labels
print("Unique categories in train set:", train_labels)
print("Unique categories in test set:", test_labels)

def balance_and_sample(data, num_samples_per_category):
    balanced_data = []
    for category in set(data['label']):
        category_samples = [item for item in data if item['label'] == category]
        balanced_samples = category_samples[:num_samples_per_category]
        balanced_data.extend(balanced_samples)
    return Dataset.from_pandas(pd.DataFrame(balanced_data))

# Balance and sample the train and test sets
train_dataset = balance_and_sample(dataset['train'], 150)
test_dataset = balance_and_sample(dataset['test'], 150)


# Check the sizes
print("\nSize of Balanced Train Set:", len(train_dataset))
print("Size of Balanced Test Set:", len(test_dataset))

In [ ]:
train_dataset

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here
from transformers import AutoTokenizer, DistilBertForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


# insert your code here
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

print("Tokenized Train Dataset:")
print(train_dataset[:5])
print("\nTokenized Test Dataset:")
print(test_dataset[:5])

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
train_dataset

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [ ]:
! pip install transformers[torch] -U


In [ ]:
from transformers import TrainingArguments, Trainer, get_scheduler
from torch.optim import AdamW

# List to store results of each configuration
all_results = []

# Define hyperparameters to experiment with
learning_rates = [1e-5, 3e-5, 5e-5]
batch_sizes = [8, 16, 32]
num_epochs = [3, 5, 10]
weight_decays = [0.01, 0.05, 0.1]

for lr in learning_rates:
    for batch_size in batch_sizes:
        for epochs in num_epochs:
            for decay in weight_decays:
                # Set up TrainingArguments
                args = TrainingArguments(
                    output_dir="test_trainer",
                    evaluation_strategy="epoch",
                    per_device_train_batch_size=batch_size,
                    learning_rate=lr,
                    num_train_epochs=epochs,
                    weight_decay=decay,
                    logging_dir='./logs',
                    save_strategy="epoch",
                    load_best_model_at_end=True,
                    metric_for_best_model="accuracy"
                )

                # Initialize Trainer without built-in optimizer/scheduler setup
                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=train_dataset,
                    eval_dataset=test_dataset,
                    compute_metrics=compute_metrics
                )

                # Custom Optimizer and Scheduler Setup
                optimizer = AdamW(model.parameters(), lr=lr, weight_decay=decay) # The optimizer is responsible for updating the model's weights based on the computed gradients.
                                                                                 # Key characteristics: Adaptive Learning Rate: AdamW adapts the learning rate for each parameter individually,
                                                                                 # making it more efficient in converging to the optimal solution.
                                                                                 # Weight Decay: Unlike traditional L2 regularization, weight decay in AdamW is decoupled from the gradient update.
                                                                                 # This means it directly subtracts a fraction of the parameter value to encourage smaller weights and prevent overfitting.
                num_training_steps = len(train_dataset) // batch_size * epochs
                scheduler = get_scheduler(
                    "linear",
                    optimizer=optimizer,
                    num_warmup_steps=0,
                    num_training_steps=num_training_steps
                ) #  A linear scheduler decreases the learning rate linearly from an initial value to a minimum value over the course of training.

                # Update trainer with custom optimizer and scheduler
                trainer.optimizer = optimizer
                trainer.lr_scheduler = scheduler

                # Train the model
                trainer.train()
                results = trainer.evaluate()

                # Save results for each configuration
                result_data = {
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "num_epochs": epochs,
                    "weight_decay": decay,
                    "accuracy": results["eval_accuracy"]
                }
                all_results.append(result_data)

# Print all results
for result in all_results:
    print(result)


# Model Training Results

| Learning Rate | Batch Size | Num Epochs | Weight Decay | Accuracy |
|:-------------:|:----------:|:----------:|:------------:|:--------:|
| 1e-05         | 8          | 3          | 0.01         | 0.92     |
| 1e-05         | 8          | 3          | 0.05         | 0.93     |
| 1e-05         | 8          | 3          | 0.1          | 0.9133   |
| 1e-05         | 8          | 5          | 0.01         | 0.9067   |
| 1e-05         | 8          | 5          | 0.05         | 0.9067   |
| 1e-05         | 8          | 5          | 0.1          | 0.92     |
| 1e-05         | 8          | 10         | 0.01         | 0.92     |
| 1e-05         | 8          | 10         | 0.05         | 0.9267   |
| 1e-05         | 8          | 10         | 0.1          | 0.9233   |
| 1e-05         | 16         | 3          | 0.01         | 0.9233   |
| 1e-05         | 16         | 3          | 0.05         | 0.9233   |
| 1e-05         | 16         | 3          | 0.1          | 0.9233   |
| 1e-05         | 16         | 5          | 0.01         | 0.9233   |
| 1e-05         | 16         | 5          | 0.05         | 0.92     |
| 1e-05         | 16         | 5          | 0.1          | 0.92     |
| 1e-05         | 16         | 10         | 0.01         | 0.92     |
| 1e-05         | 16         | 10         | 0.05         | 0.92     |
| 1e-05         | 16         | 10         | 0.1          | 0.92     |
| 1e-05         | 32         | 3          | 0.01         | 0.92     |
| 1e-05         | 32         | 3          | 0.05         | 0.9167   |
| 1e-05         | 32         | 3          | 0.1          | 0.92     |
| 1e-05         | 32         | 5          | 0.01         | 0.92     |
| 1e-05         | 32         | 5          | 0.05         | 0.9233   |
| 1e-05         | 32         | 5          | 0.1          | 0.9233   |
| 1e-05         | 32         | 10         | 0.01         | 0.9233   |
| 1e-05         | 32         | 10         | 0.05         | 0.9233   |
| 1e-05         | 32         | 10         | 0.1          | 0.9233   |
| 3e-05         | 8          | 3          | 0.01         | 0.92     |
| 3e-05         | 8          | 3          | 0.05         | 0.92     |
| 3e-05         | 8          | 3          | 0.1          | 0.9167   |
| 3e-05         | 8          | 5          | 0.01         | 0.92     |
| 3e-05         | 8          | 5          | 0.05         | 0.92     |
| 3e-05         | 8          | 5          | 0.1          | 0.92     |
| 3e-05         | 8          | 10         | 0.01         | 0.9233   |
| 3e-05         | 8          | 10         | 0.05         | 0.9233   |
| 3e-05         | 8          | 10         | 0.1          | 0.9233   |
| 3e-05         | 16         | 3          | 0.01         | 0.9233   |
| 3e-05         | 16         | 3          | 0.05         | 0.9233   |
| 3e-05         | 16         | 3          | 0.1          | 0.9233   |
| 3e-05         | 16         | 5          | 0.01         | 0.92     |
| 3e-05         | 16         | 5          | 0.05         | 0.92     |
| 3e-05         | 16         | 5          | 0.1          | 0.92     |
| 3e-05         | 16         | 10         | 0.01         | 0.92     |
| 3e-05         | 16         | 10         | 0.05         | 0.92     |
| 3e-05         | 16         | 10         | 0.1          | 0.92     |
| 3e-05         | 32         | 3          | 0.01         | 0.92     |
| 3e-05         | 32         | 3          | 0.05         | 0.92     |
| 3e-05         | 32         | 3          | 0.1          | 0.92     |
| 3e-05         | 32         | 5          | 0.01         | 0.9233   |
| 3e-05         | 32         | 5          | 0.05         | 0.9033   |
| 3e-05         | 32         | 5          | 0.1          | 0.9033   |
| 3e-05         | 32         | 10         | 0.01         | 0.9033   |
| 3e-05         | 32         | 10         | 0.05         | 0.9033   |
| 3e-05         | 32         | 10         | 0.1          | 0.9033   |
| 5e-05         | 8          | 3          | 0.01         | 0.9033   |
| 5e-05         | 8          | 3          | 0.05         | 0.9033   |
| 5e-05         | 8          | 3          | 0.1          | 0.9033   |
| 5e-05         | 8          | 5          | 0.01         | 0.9033   |
| 5e-05         | 8          | 5          | 0.05         | 0.9033   |
| 5e-05         | 8          | 5          | 0.1          | 0.9033   |
| 5e-05         | 8          | 10         | 0.01         | 0.9033   |
| 5e-05         | 8          | 10         | 0.05         | 0.9033   |
| 5e-05         | 8          | 10         | 0.1          | 0.9067   |
| 5e-05         | 16         | 3          | 0.01         | 0.9067   |
| 5e-05         | 16         | 3          | 0.05         | 0.9067   |
| 5e-05         | 16         | 3          | 0.1          | 0.9067   |
| 5e-05         | 16         | 5          | 0.01         | 0.9067   |
| 5e-05         | 16         | 5          | 0.05         | 0.9067   |
| 5e-05         | 16         | 5          | 0.1          | 0.9067   |
| 5e-05         | 16         | 10         | 0.01         | 0.9067   |
| 5e-05         | 16         | 10         | 0.05         | 0.9067   |
| 5e-05         | 16         | 10         | 0.1          | 0.9067   |
| 5e-05         | 32         | 3          | 0.01         | 0.9067   |
| 5e-05         | 32         | 3          | 0.05         | 0.9067   |
| 5e-05         | 32         | 3          | 0.1          | 0.9067   |
| 5e-05         | 32         | 5          | 0.01         | 0.91     |
| 5e-05         | 32         | 5          | 0.05         | 0.91     |
| 5e-05         | 32         | 5          | 0.1          | 0.91     |
| 5e-05         | 32         | 10         | 0.01         | 0.91     |
| 5e-05         | 32         | 10         | 0.05         | 0.91     |
| 5e-05         | 32         | 10         | 0.1          | 0.91     |


## Ανάλυση της Επίδρασης των Υπερπαραμέτρων στo fine tuning του μοντέλου

### Ρυθμός Μάθησης (Learning Rate)
- **1e-05**: Αυτός ο ρυθμός μάθησης φαίνεται να παρέχει σταθερή και υψηλή ακρίβεια σε διαφορετικά batch sizes, αριθμό εποχών και τιμές weight decay. Οι ακρίβειες κυμαίνονται από 0.9033 έως 0.93, με υψηλότερες ακρίβειες να παρατηρούνται συνήθως σε μέτριες τιμές weight decay (0.05).
- **3e-05**: Παρόμοια, αυτός ο ρυθμός μάθησης διατηρεί υψηλά επίπεδα ακρίβειας, αν και παρουσιάζει μικρή πτώση περίπου στο 0.9033 σε ορισμένες διαμορφώσεις με μεγαλύτερα batch sizes και υψηλότερες τιμές weight decay.
- **5e-05**: Αυτός ο υψηλότερος ρυθμός μάθησης δείχνει σταθερά ελαφρώς χαμηλότερες ακρίβειες σε σύγκριση με τους άλλους δύο ρυθμούς μάθησης, ιδίως για μεγαλύτερα batch sizes και υψηλότερες τιμές weight decay, όπου οι ακρίβειες τείνουν να κυμαίνονται γύρω στο 0.9033 έως 0.91.

### Batch Size
- **8**: Αυτό το batch size επιτυγχάνει την υψηλότερη ακρίβεια (έως 0.93), αλλά δείχνει κάποια ευαισθησία στις αλλαγές του weight decay και του ρυθμού μάθησης.
- **16**: Γενικά διατηρεί υψηλή και σταθερή ακρίβεια, συχνά γύρω στο 0.92 έως 0.9233 (εκτός από τον υψηλότερο ρυθμό μάθησης), υποδεικνύοντας μια καλή ισορροπία μεταξύ σταθερότητας και απόδοσης.
- **32**: Δείχνει σταθερή απόδοση, αλλά παρατηρούνται ελαφρές μειώσεις στην ακρίβεια (περίπου 0.9033 έως 0.9233) με υψηλότερες τιμές weight decay και ρυθμούς μάθησης.

## Αριθμός Εποχών (Num Epochs)

- **3 Εποχές**:
  - Αυτή η επιλογή παρέχει αρχικά αποτελέσματα υψηλής ακρίβειας, με τις ακρίβειες να κυμαίνονται από 0.9033 έως 0.9233.
  - Ωστόσο, οι ακρίβειες είναι για κάποιους συνδυασμούς παραμέτρων χαμηλότερες σε σύγκριση με περισσότερες εποχές, υποδεικνύοντας ότι το μοντέλο μπορεί να μην έχει εκπαιδευτεί πλήρως.

- **5 Εποχές**:
  - Με 5 εποχές, παρατηρούμε αύξηση της ακρίβειας σε πολλές διαμορφώσεις.
  - Παράδειγμα: Για ρυθμό μάθησης 1e-05, batch size 8 και weight decay 0.1, οι ακρίβειες αυξάνονται από 0.9133 (3 εποχές) σε 0.92(5 εποχές), κάτι που δείχνει ότι το επιπλέον εκπαίδευση προσφέρει βελτίωση, αν και μικρή.
  - Παρόλα αυτά, σε ορισμένες διαμορφώσεις, η αύξηση των εποχών δεν προσφέρει σημαντική βελτίωση, υποδεικνύοντας ότι υπάρχει μια βέλτιστη διάρκεια εκπαίδευσης για κάθε διαμόρφωση.

- **10 Εποχές**:
  - Οι περισσότερες διαμορφώσεις που χρησιμοποιούν 10 εποχές δείχνουν τις υψηλότερες ακρίβειες, όπως 0.9233 και άνω.
  - Αυτό υποδεικνύει ότι οι περισσότερες εποχές παρέχουν στο μοντέλο τον απαραίτητο χρόνο για πλήρη εκπαίδευση και βελτίωση.
  - Σε μερικές περιπτώσεις, η ακρίβεια παραμένει σταθερή ή μειώνεται ελαφρώς (0.91). Αυτό υποδεικνύει ότι το πολύ μεγάλο πλήθος εποχών μπορεί να μην προσφέρει επιπλέον βελτίωση ή να οδηγήσει σε υπερπροσαρμογή.

### Weight Decay
- **0.01**: Παρέχει σταθερή και υψηλή ακρίβεια σε διαφορετικές διαμορφώσεις, υποδεικνύοντας αποτελεσματική κανονικοποίηση χωρίς υπερβολική ποινή στην πολυπλοκότητα του μοντέλου.
- **0.05**: Γενικά οδηγεί σε ελαφρώς υψηλότερες ακρίβειες σε σύγκριση με το 0.01, υποδεικνύοντας βέλτιστη κανονικοποίηση για αυτό το σύνολο δεδομένων και διαμόρφωση μοντέλου.
- **0.1**: Οι ακρίβειες τείνουν να μειώνονται ελαφρώς, ειδικά με υψηλότερους ρυθμούς μάθησης και batch sizes, υποδεικνύοντας πιθανή υπερ-κανονικοποίηση.

### Σύνοψη
- **Βέλτιστη Διαμόρφωση**: Βάσει αυτών των αποτελεσμάτων, ένας ρυθμός μάθησης 1e-05 , με batch size 8 και weight decay 0.05, εκπαιδευμένο σε 3 εποχές, φαίνεται να προσφέρει την καλύτερη απόδοση βελτίωσης.
- **Ευαισθησία**: Το μοντέλο δείχνει ευαισθησία σε υψηλότερους ρυθμούς μάθησης και τιμές weight decay, ιδίως με μεγαλύτερα batch size, υποδεικνύοντας την ανάγκη για προσεκτική εξισορρόπηση αυτών των υπερπαραμέτρων.



# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
from datasets import load_dataset, Dataset
import random

# # insert your code here (load dataset)
piqa_dataset = load_dataset("piqa")

train_data = piqa_dataset["train"]

random_sample = random.sample(range(len(train_data)), 100)

sampled_data = [train_data[i] for i in random_sample]

for i, sample in enumerate(sampled_data):
    print(f"Sample {i+1}:")
    print(f"Question: {sample['goal']}")
    print(f"Choice 1: {sample['sol1']}")
    print(f"Choice 2: {sample['sol2']}")
    print(f"Label: {sample['label']}")
    print()

sampled_dataset = Dataset.from_list(sampled_data)


Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
# insert your code here (models)
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments
import torch

model_names = [
    "bert-large-uncased",
    "roberta-large",
    "nghuyong/ernie-2.0-en",
    "xlnet-base-cased",
    "albert-large-v2"
]

# Initialize tokenizers and models
tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in model_names]
models = [AutoModelForMultipleChoice.from_pretrained(model_name) for model_name in model_names]



In [ ]:
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
def preprocess_function(examples, tokenizer):
    first_sentences = [[context] * 2 for context in examples["goal"]]

    # A list where each element is a tuple containing corresponding elements from examples["sol1"] and examples["sol2"].
    question_headers = list(zip(examples["sol1"], examples["sol2"]))

    # A list of lists, where each inner list contains the string representations of the elements from each tuple in question_headers.
    second_sentences = [list(map(str, header)) for header in question_headers]

    # Flatten the inputs
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True, padding=True)
    return {k: [v[i:i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [ ]:
for model_name, tokenizer, model in zip(model_names, tokenizers, models):
    encoded_dataset = sampled_dataset.map(lambda examples: preprocess_function(examples, tokenizer), batched=True)

    training_args = TrainingArguments(
        output_dir='./results',
        per_device_eval_batch_size=8,
        do_train=False,
        do_eval=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        eval_dataset=encoded_dataset,
        compute_metrics=compute_metrics
    )

    print(f"Evaluating model: {model_name}")
    results = trainer.evaluate()
    print(f"Accuracy: {results['eval_accuracy']:.4f}")

## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
from datasets import load_dataset
import random

dataset = load_dataset('truthful_qa', "generation", split='validation')

filtered_samples = []
for sample in dataset:
    if len(sample['correct_answers']) >= 2:
        filtered_samples.append(sample)

if len(filtered_samples) < 100:
    raise ValueError("Not enough samples with at least 2 correct answers.")

selected_samples = random.sample(filtered_samples, 100)

final_samples = []
for sample in selected_samples:
    best_answer = sample['best_answer']
    correct_answers = sample['correct_answers'][:2]

    # Choose a random incorrect answer
    incorrect_answer = random.choice(sample['incorrect_answers'])

    options = [best_answer, correct_answers[0], correct_answers[1], incorrect_answer]

    final_sample = {
        'question': sample['question'],
        'options': options
    }

    final_samples.append(final_sample)

# Display the final samples
for i, sample in enumerate(final_samples[:20], 1):
    print(f"Sample {i}:")
    print(f"Question: {sample['question']}")
    for idx, option in enumerate(sample['options']):
        print(f"Option {idx + 1}: {option}")
    print()


In [ ]:
from sentence_transformers import SentenceTransformer

semantic_model_names = [
    'sentence-transformers/all-MiniLM-L6-v2',
    'sentence-transformers/all-mpnet-base-v2',
    'sentence-transformers/all-MiniLM-L12-v2',
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    'sentence-transformers/bert-base-nli-mean-tokens',
    'sentence-transformers/multi-qa-MiniLM-L6-cos-v1'
]

semantic_models = [(name, SentenceTransformer(name)) for name in semantic_model_names]


In [ ]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments
import torch

qa_model_names = [
    "bert-large-uncased",
    "roberta-large",
    "nghuyong/ernie-2.0-en",
    "xlnet-base-cased",
    "albert-large-v2"
]

qa_tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in qa_model_names]
qa_models = [AutoModelForMultipleChoice.from_pretrained(model_name) for model_name in qa_model_names]



In [ ]:
def preprocess_function(examples, tokenizer):
    first_sentences = [[question] * 4 for question in examples['question']]
    options = examples['options']

    second_sentences = []
    for option_list in options:
        second_sentences.extend(option_list)

    # Flatten the inputs
    first_sentences = [item for sublist in first_sentences for item in sublist]

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True, padding=True)
    return {k: [v[i:i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

encoded_datasets = [
    Dataset.from_list(final_samples).map(lambda examples: preprocess_function(examples, tokenizer), batched=True)
    for tokenizer in qa_tokenizers
]


In [ ]:
def compute_accuracy(predictions, references, semantic_model):
    correct = 0
    threshold = 0.95

    for pred, ref in zip(predictions, references):
        best_answer = ref[0]
        correct_answers = ref[1:3]
        predicted_answer = ref[pred]

        if predicted_answer == best_answer:
          correct += 1

        elif predicted_answer in correct_answers:
            best_embedding = semantic_model.encode([best_answer])[0]
            predicted_embedding = semantic_model.encode([predicted_answer])[0]
            similarity = get_cosine_similarity(best_embedding, predicted_embedding)
            if similarity > threshold:
                correct += 1

    return correct / len(predictions)

reference_labels = [
    [sample['options'][0], sample['options'][1], sample['options'][2], sample['options'][3]]
    for sample in final_samples
]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True
)

def print_formatted_results(model_name, results):
    print(f"\nEvaluating Model: {model_name}")
    print("="*50)
    for semantic_model_name, accuracy in results.items():
        print(f"Semantic Model: {semantic_model_name}")
        print(f"Accuracy: {accuracy:.4f}")
        print("-" * 50)

for model_name, tokenizer, model, encoded_dataset in zip(qa_model_names, qa_tokenizers, qa_models, encoded_datasets):
    results = {}

    trainer = Trainer(
        model=model,
        args=training_args,
        eval_dataset=encoded_dataset
    )

    raw_predictions = trainer.predict(encoded_dataset)
    predictions = torch.argmax(torch.tensor(raw_predictions.predictions), dim=-1)

    for semantic_model_name, semantic_model in semantic_models:
        accuracy = compute_accuracy(predictions, reference_labels, semantic_model)
        results[semantic_model_name] = accuracy

    print_formatted_results(model_name, results)

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
# insert your code here (load dataset)
from datasets import load_dataset
from transformers import pipeline
import random

dataset = load_dataset('winogrande', 'winogrande_xl', split='validation')

sample_size = 100
random_sample = random.sample(list(dataset), sample_size)

for item in random_sample:
    print(item)


Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [ ]:
# insert your code here (load models)
model_names = [
    "DeepPavlov/roberta-large-winogrande",
    "roberta-large-mnli",
    "bert-large-uncased"
]

In [ ]:
# insert your code here (create pipelines)
pipelines = {model_name: pipeline("text-classification", model=model_name) for model_name in model_names}


In [ ]:
def predict_answer(model_pipelines, sample):
    predictions = {}
    for model_name, model_pipeline in model_pipelines.items():
        sentence = sample['sentence']
        option1 = sample['option1']
        option2 = sample['option2']

        # Create two versions of the sentence with each option
        input1 = sentence.replace('_', option1)
        input2 = sentence.replace('_', option2)

        # Get classification scores for each option
        result1 = model_pipeline(input1)
        result2 = model_pipeline(input2)

        score1 = result1[0]['score']
        score2 = result2[0]['score']

        # Choose the option with the higher score
        predictions[model_name] = '1' if score1 > score2 else '2'
    return predictions


# Initialize counters for correct predictions
correct_predictions = {model_name: 0 for model_name in model_names}

# Predict and count correct answers
for item in random_sample:
    predictions = predict_answer(pipelines, item)
    print(f"Sentence: {item['sentence']}")
    print(f"Option 1: {item['option1']}")
    print(f"Option 2: {item['option2']}")
    print(f"True Answer: {item['answer']}")
    for model_name, prediction in predictions.items():
        print(f"Model: {model_name}, Prediction: {prediction}")
    true_answer = item['answer']
    for model_name, prediction in predictions.items():
        if prediction == true_answer:
            correct_predictions[model_name] += 1

# Calculate and print accuracy
for model_name in model_names:
    accuracy = correct_predictions[model_name] / sample_size
    print(f"Model: {model_name}, Accuracy: {accuracy:.2f}")